This notebook based on this link: https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

In [ ]:
#Reading the data
import re
import pandas as pd

df = pd.read_csv('turkce_haber_4k.csv')
df.head()

,Unnamed: 0,Text,Summary
0,0,"\nIğdır'da bir kişi, tartıştığı karısını silah...","Iğdır'da, Umut adlı erkek evi terk eden 1 çocu..."
1,1,\nCumhurbaşkanı Recep Tayyip Erdoğan'ın katılı...,Karsan Otomotiv'den yapılan açıklamada iş insa...
2,2,"\nTürkiye'nin Otomobili'nin, Bilişim Vadisi'nd...","Türkiye'nin Otomobili'nin, Bilişim Vadisi'nde ..."
3,3,\nVan'ın Erciş ilçesinde evden çıktıktan sonra...,Van'ın Erciş ilçesinde evden çıktıktan sonra b...
4,4,\nSon dönemin parlayan yıldızlarından olan baş...,"Dünyaca ünlü şarkıcı Rita Ora, annesi Vera Ora..."


In [ ]:
df.shape

(4484, 3)

In [ ]:
#Rename the column names
df=df.rename(columns={"Text": "ctext", "Summary": "text"})

In [ ]:
#Drop unnecessary column
df=df.drop(columns=['Unnamed: 0'])

In [ ]:
#Data cleaning
df['text']=df['text'].apply(lambda x:' '.join(re.sub(r'(\s)##\w+##', r'\1', x).split()))
df['ctext']=df['ctext'].apply(lambda x:' '.join(re.sub(r'(\s)##\w+##', r'\1', x).split()))

In [ ]:
df['text'][0], df['ctext'][0]

("Iğdır'da, Umut adlı erkek evi terk eden 1 çocuk annesi eşini tabancayla vurarak öldürdü, ardından kendisini göğsünden vurdu. Yaralanan süpheli, hastanede tedaviye alındı.",
 "Iğdır'da bir kişi, tartıştığı karısını silahla vurarak öldürdükten sonra intihar girişiminde bulundu. EVİ TERK EDEN EŞİNİ IĞDIR'DA BULDUOlay, saat 18.00 sıralarında Kazım Karabekir Caddesi'nde meydana geldi. Ağrı'nın Doğubayazıt ilçesinde oturan 1 çocuk annesi Ebru A., yaklaşık 3 ay önce henüz bilinmeyen nedenle eşini terk edip, Iğdır'a geldi. Burada ev kiralayan Ebru A.'yı bulan eşi Umut A. kente geldi. Eşinin evine giden Umut A., kapıyı çalıp, içeri girdi. ARALARINDA TARTIŞMA ÇIKTI, EŞİNE ATEŞ ETTİBu sırada iddiaya göre, ikili arasında tartışma çıktı. Tartışmanın büyümesi üzerine Umut A., yanında getirdiği tabanca ile Ebru A.'ya ateş etti. Ebru A. kanlar içinde yere yığılırken, Umut A. kendisini de göğsünden vurdu. KADIN EVDE ÖLDÜ, ŞÜPHELİ HASTANEDE TEDAVİ GÖRÜYORSilah sesini duyan komşuların ihbarı üzerine ol

In [ ]:
df['text'][2], df['ctext'][2]

("Türkiye'nin Otomobili'nin, Bilişim Vadisi'nde açılış töreni organizasyonunu gerçekleştiren YRD İstanbul'un Yönetim Kurulu Başkanı Fikret Karadeniz, yerli otomobilin 7 bölgeyi gezerek vatandaşlarla buluşacağını söyledi.",
 'Türkiye\'nin Otomobili\'nin, Bilişim Vadisi\'ndeki açılış töreni organizasyonunu gerçekleştiren YRD İstanbul\'un Yönetim Kurulu Başkanı Fikret Karadeniz, Türkiye\'nin Otomobili\'nin ön gösterim araçlarının henüz vatandaşlarla buluşması için erken olduğunu belirterek, "Marka lansmanı daha sonraki bir aşama ve marka lansmanıyla birlikte bildiğim kadarıyla araçlar 7 bölgeyi dolaşacak ve vatandaşlarla buluşturulacak." dedi. "AYNI HEYECANI BİZ DE YAŞAMAYA BAŞLADIK"Türkiye\'nin Otomobili ön gösterim aracı tanıtımı projesine yaklaşık 1,5 ay önce Sanayi ve Teknoloji Bakanı Mustafa Varank\'ın talimatıyla başladıklarını aktaran Karadeniz, "TOGG ekibiyle ben daha öncesinde tanışmıyordum. TOGG CEO\'su Gürcan Bey ve ekibiyle tanışıp onların profesyonel yaklaşımlarını, heyecanla

In [ ]:
# Check the words in sentences
(len(df['ctext'][15].split(' '))), (len(df['text'][15].split(' ')))

(220, 23)

In [ ]:
#Save the csv file to data
df.to_csv('temiz_turkce_haber_4k.csv')

In [ ]:
!pip install transformers -q
!pip install wandb -q
!pip install sentencepiece
#!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

     |████████████████████████████████| 2.3MB 7.4MB/s 
     |████████████████████████████████| 3.3MB 52.1MB/s 
     |████████████████████████████████| 901kB 53.2MB/s 
     |████████████████████████████████| 1.8MB 6.8MB/s 
     |████████████████████████████████| 174kB 37.1MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 133kB 28.7MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 1.2MB 7.4MB/s 


In [ ]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# WandB – Import the wandb library
import wandb

In [ ]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Wed May 26 21:29:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

#import torch_xla
#import torch_xla.core.xla_model as xm
#device = xm.xla_device()

In [ ]:
# Login to wandb to log the model run and all the parameters
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach() # lm_labels instead of labels
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
def main():
    # WandB – Initialize a new run
    wandb.init(project="transformers_T5_fine_tuning")

    # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training  
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 1 
    config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 512           # 512
    config.SUMMARY_LEN = 150        # 150
    #truncation=True
    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
    df = pd.read_csv('temiz_turkce_haber_4k.csv')
    df = df[['text','ctext']]
    df.ctext = 'summarize: ' + df.ctext
    print(df.head())

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    # Log metrics with wandb
    wandb.watch(model, log="all")
    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(config.VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('predictions.csv')
        print('Output Files generated for review')


    # Saving Fine Tuned Model
    output_dir = '/content'

    print("Saving model to %s" % output_dir)

    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

if __name__ == '__main__':
    main()

Training Loss,1.13651
_runtime,1827
_timestamp,1622067451
_step,360


Training Loss,▅▆▅█▄▆▅▆▆▄▅▅▄▂▅▄▄▂▂▅▁▂▂▂▃▂▃▁▂▁▃▁▁▂▁▃▂▂▂▃
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


                                                text                                              ctext
0  Iğdır'da, Umut adlı erkek evi terk eden 1 çocu...  summarize: Iğdır'da bir kişi, tartıştığı karıs...
1  Karsan Otomotiv'den yapılan açıklamada iş insa...  summarize: Cumhurbaşkanı Recep Tayyip Erdoğan'...
2  Türkiye'nin Otomobili'nin, Bilişim Vadisi'nde ...  summarize: Türkiye'nin Otomobili'nin, Bilişim ...
3  Van'ın Erciş ilçesinde evden çıktıktan sonra b...  summarize: Van'ın Erciş ilçesinde evden çıktık...
4  Dünyaca ünlü şarkıcı Rita Ora, annesi Vera Ora...  summarize: Son dönemin parlayan yıldızlarından...
FULL Dataset: (4484, 2)
TRAIN Dataset: (3587, 2)
TEST Dataset: (897, 2)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset
Epoch: 0, Loss:  5.8043365478515625


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  2.1652960777282715
Epoch: 0, Loss:  1.3832365274429321
Epoch: 0, Loss:  0.4993664026260376
Epoch: 1, Loss:  0.7837203145027161
Epoch: 1, Loss:  0.7894322872161865
Epoch: 1, Loss:  0.7884302735328674
Epoch: 1, Loss:  0.7475789785385132
Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe
Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Output Files generated for review
Saving model to /content


In [ ]:
import pandas as pd

pred = pd.read_csv('predictions.csv')
pred.head()

,Unnamed: 0,Generated Text,Actual Text
0,0,Tekneyle çktklar deniz turunun keyfini çkard. ...,"Dünyaca ünlü şarkc Rita Ora, annesi Vera Ora i..."
1,1,"Yeliz Kuvanc, 'Tutunamayanlar' dizide başrolle...","Güzel oyuncu Yeliz Kuvanc, TRT'nin iddial absü..."
2,2,"Ankara Valisi Vasip ahin, devam eden kar yaşnn...","Ankara Valisi Vasip ahin, kentte devam eden ka..."
3,3,Sivri dilinden dolay girdii polemiklerle sk sk...,Cumhurbaşkan Recep Tayyip Erdoan' ziyaret eden...
4,4,"arkc, 55 bin TL marka çantasn modelleri ilk ke...","Son dönemin çok konuşulan ismi Aleyna Tilki, 5..."


In [ ]:
pred['Generated Text'][1], pred['Actual Text'][1]

("Yeliz Kuvanc, 'Tutunamayanlar' dizide başrollerini paylaşt dizide, Tark rolünü canlandran ünlü oyuncu Dou Demirkol'un ablas Ayşe karakteriyle izleyiciyle buluşmaya hazrlanyor.",
 "Güzel oyuncu Yeliz Kuvanc, TRT'nin iddial absürt komedisi olan Tutunamayanlar ile ekranlara dönüyor. Kuvanc dizide başrol oyuncusu Dou Demirkol'un ablas Ayşe karakterini canlandracak.")

In [ ]:
# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Loading Fine Tuned Model
output_dir = '/content'

# Load a trained model and vocabulary that you have fine-tuned
model = T5ForConditionalGeneration.from_pretrained(output_dir)
tokenizer = T5Tokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
#model = model.to(device)

In [ ]:
#Summarize function
def summarize(text, max_length=150):
  
  input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

  generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)

  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

  return preds[0]

In [ ]:
#Testing
summarize("""
Galatasaray'da tarihi seçime sayılı günler kala eski başkan Ünal Aysal'dan çarpıcı bir hamle geldi. \n
 Birçok başkan adayının çıkması sonrası Aysal, Adnan Öztürk ve Sedat Doğan'ında katılacağı bir toplantı yapma kararı aldı. \n
 Burak Elmas ve Eşref Hamamcıoğlu'nun da davet edileceği toplantı da Aysal'ın adaylara birleşme teklifi yapması bekleniyor. \n
""")

"başkan adaynn çkmas sonras Aysal, Burak Elmas ve Sedat Doan'nda katlaca bir toplant yapma karar ald. Galatasaray'da tarihi seçime sayl günler kala eski başkan çarpc bir hamle geldi."